In [1]:
import time
import numpy as np
import pandas as pd
import xgboost as xgb
from sklearn import linear_model
from sklearn.cross_validation import train_test_split
from sklearn.ensemble import RandomForestRegressor
#from sklearn import pipeline, model_selection
from sklearn import pipeline, grid_search
#from sklearn.feature_extraction import DictVectorizer
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import FeatureUnion
from sklearn.decomposition import TruncatedSVD
#from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import mean_squared_error, make_scorer

import re

import random
random.seed(2016)
pd.set_option("display.max_colwidth", 200)

# LOC = '/home/ec2-user/data/hd/unpacked/'
# df_train = pd.read_csv(LOC + 'train.csv', encoding="ISO-8859-1")
# num_train = df_train.shape[0]

In [2]:
dtrain = xgb.DMatrix("train.buffer")
dtest = xgb.DMatrix("test.buffer")
evallist  = [(dtrain,'train')]

In [3]:
a = np.load('train_data.npy')
b = np.load('test_data.npy')
a_brand = np.load('features_brand_01_train.npy')
b_brand = np.load('features_brand_01_test.npy')
a_other = np.load('FEATURES_1d_TRAIN.npy')
b_other = np.load('FEATURES_1d_TEST.npy')
a = np.hstack((a, a_brand, a_other))
b = np.hstack((b, b_brand, b_other))

In [4]:
X_train, X_test, y_train, y_test = train_test_split(a, dtrain.get_label(), test_size=0.20, random_state=147)
gX_train = xgb.DMatrix(data=X_train, label=y_train)
gX_test = xgb.DMatrix(data=X_test, label=y_test)
evallist  = [(gX_train,'train'),(gX_test,'test')]

In [5]:
def fmean_squared_error(ground_truth, predictions):
    fmean_squared_error_ = mean_squared_error(ground_truth, predictions)**0.5
    return fmean_squared_error_

In [6]:
#clf = linear_model.LinearRegression(n_jobs=8)
clf = linear_model.Ridge (alpha = .6)
clf.fit(X_train, y_train)
y_hat = clf.predict(X_test)
y_hat = np.minimum(np.maximum(y_hat, 1.0), 3.0)
fmean_squared_error(y_hat, y_test)

0.47253216191863778

In [7]:
# TRAIN AS SELF
param = {'max_depth':9, 
         'eta':0.1, # 'objective':'reg:linear',
         'eval_metric':'rmse', #'maximize': False,
         'colsample_bytree':0.3,
         'subsample':0.9,
         'nthread':8,
         'silent': True
        }
# GOOD
param = {'max_depth':7, 
         'eta':0.05, # 'objective':'reg:linear',
         'eval_metric':'rmse', #'maximize': False,
         'colsample_bytree':0.8, #7
         'subsample':0.9,  #8
         #'min_child_weight': 4.0,
         'nthread':16,
         'silent': True
        }
num_round = 5000
bst = xgb.train( param, gX_train, num_round, [(gX_train,'train'),(gX_test,'test')], early_stopping_rounds=15, verbose_eval=10)